In [36]:
#Author - Kovid Sharma
#zID - z5240067

import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

def GetTime(seconds):
    h = int(seconds//(60*60))
    m = int((seconds-h*60*60)//60)
    s = seconds-(h*60*60)-(m*60)
    if h == 0 and m == 0:
        return str(f'{round(s,1)} secs')
    if h == 0:
        return str(f'{m}:{round(s,1)} mins')
    if m == 0:
        return str(f'{round(s,1)} secs')
    return str(f'{h} hours {m}:{round(s,1)} mins')
#----------------------------------------------------------------------------#
def histogram(I):
    xaxis, intensities = np.unique(I, return_counts=True)
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.bar(xaxis, intensities)
    plt.show()
    #to show x-axis labels
#     plt.bar(xaxis, intensities)
#     plt.xticks(xaxis, fontsize=5, rotation=90)
#     plt.show()
#----------------------------------------------------------------------------#    
def contrast_stretch(I):
    a = 0
    b = 255
    c = np.amin(I)
    d = np.amax(I)    
    #contrast stretched
    O = ( (I - c) * ((b-a)/(d-c)) ) + a
    return O
#----------------------------------------------------------------------------#
def image_edge(I):
    Sx = np.array([[-1, 0, 1], [-2, 0, 2],[-1, 0, 1]], dtype='uint')
    Sy = np.array([[-1,-2,-1], [ 0, 0, 0],[-1, 2, 1]], dtype='uint')
    Ox = I 
    Oy = I 
    I_G = cv2.GaussianBlur(I,(3,3),0)
    laplacian = cv2.Laplacian(I_G,cv2.CV_64F)
    Sobelx = cv2.Sobel(I_G, cv2.CV_64F, 1, 0, ksize = 3)
    Sobely = cv2.Sobel(I_G, cv2.CV_64F, 0, 1, ksize = 3)
    O =  (cv2.add (cv2.multiply(Sobelx, Sobelx), cv2.multiply(Sobely, Sobely)) ** 0.5)
#     cv2.imwrite('cat_3_gaussian.png',I_G)
#     cv2.imwrite('cat_3_laplacian.png',laplacian)
#     cv2.imwrite('cat_3_Sobelx.png',Sobelx)
#     cv2.imwrite('cat_3_Sobely.png',Sobely)
#     cv2.imwrite('cat_3_edges.png',O)
    return I_G, laplacian, Sobelx, Sobely, O
#----------------------------------------------------------------------------#    
def sharpening(I, kernel, sigma, a):
    L = cv2.GaussianBlur(I, kernel, sigma)
    L = L.astype('int32')
    H = I - L
    H = H.astype('int32')
    Ha = H * a
    Ha = Ha.astype('int32')
    S = I + Ha
    S = S.astype('int32')
#     cv2.imwrite('cat_Q4_L_Sigma_'+str(sigma)+'.png',L)
#     cv2.imwrite('cat_Q4_H_Sigma_'+str(sigma)+'.png',H)
#     cv2.imwrite('cat_Q4_Ha_Sigma_'+str(sigma)+'.png',Ha)
#     cv2.imwrite('cat_Q4_S_Sigma_'+str(sigma)+'.png',S)
    return L, H, Ha, S
#----------------------------------------------------------------------------#
def q1(I):
    #Question 1
    O = contrast_stretch(I)
#     cv2.imwrite('cat_Q1_contrast_stretch.png',O)
    #plotting images
    fig, axs = plt.subplots(1,2,figsize=(15,15)) #increase window size
    #title for plot
    fig.tight_layout(rect=[0, 0, 1, 1])
    fig.suptitle('Lab 1 : Question 1', fontsize=16)
    plt.subplot(2,2,1),plt.imshow(I,cmap = 'gray')
    plt.title('Input'), plt.xticks([]), plt.yticks([])
    plt.subplot(2,2,2),plt.imshow(O,cmap = 'gray')
    plt.title('Output'), plt.xticks([]), plt.yticks([])
    plt.show()
#    plt.savefig('Question 1.png')
#----------------------------------------------------------------------------#  
def q2(I):    
    #Question 2
    print('Lab 1 : Question 2')
    histogram(I)
    plt.hist(I.ravel(),256,[0,256]); plt.show() #inbuilt to compare
    histogram(O)
    plt.hist(O.ravel(),256,[0,256]); plt.show() #inbuilt to compare
#----------------------------------------------------------------------------#
def q3(I):
    #Question 3
    I_G, L, Sobelx, Sobely, O = image_edge(I)
    #plotting images
    fig, axs = plt.subplots(3,2,figsize=(15,15)) #increase window size
    #title for plot
    fig.tight_layout(rect=[0, 0, 1, 1])
    fig.suptitle('Lab 1 : Question 3', fontsize=16)
    plt.subplot(3,2,1),plt.imshow(I_G,cmap = 'gray')
    plt.title('Gaussian'), plt.xticks([]), plt.yticks([])  
    plt.subplot(3,2,2),plt.imshow(L,cmap = 'gray')
    plt.title('Laplacian'), plt.xticks([]), plt.yticks([])
    plt.subplot(3,2,3),plt.imshow(Sobelx,cmap = 'gray')
    plt.title('Sobelx'), plt.xticks([]), plt.yticks([])
    plt.subplot(3,2,4),plt.imshow(Sobely,cmap = 'gray')
    plt.title('Sobely'), plt.xticks([]), plt.yticks([])
    plt.subplot(3,2,5),plt.imshow(I,cmap = 'gray')
    plt.title('Input'), plt.xticks([]), plt.yticks([])
    plt.subplot(3,2,6),plt.imshow(O,cmap = 'gray')
    plt.title('sqrt(Sobelx**2 + Sobely **2)'), plt.xticks([]), plt.yticks([])
    plt.savefig('Question 3.png')
    plt.show()
#----------------------------------------------------------------------------#
def q4(I):
    #Question 4
    for sigma in range(1,6,2):
        a = 1.25
        kernel = (5, 5)
        L, H, Ha, S = sharpening(I, kernel, sigma, a)
#         print('S Min:', np.min(S))
#         print('S Max:', np.max(S))
        O = cv2.normalize(S, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
        cv2.imwrite('cat_Q4_O_Sigma_'+str(sigma)+'_normalised.png',O)
#         print('O Min:', np.min(O))
#         print('O Max:', np.max(O))
#         print()

#     print('I')
#     plt.hist(I.ravel(),256,[0,256]); plt.show()
#     print('S')
#     plt.hist(S.ravel(),256,[0,256]); plt.show()
#     print('O')
#     plt.hist(O.ravel(),256,[0,256]); plt.show()
    
#     #plotting images
#     fig, axs = plt.subplots(1,2,figsize=(15,15)) #increase window size
#     #title for plot
#     fig.tight_layout(rect=[0, 0, 1, 1])
#     fig.suptitle('Lab 1 : Question 4', fontsize=16)
#     plt.subplot(1,2,1),plt.imshow(I,cmap = 'gray')
#     plt.title('Input'), plt.xticks([]), plt.yticks([])  
#     plt.subplot(3,2,2),plt.imshow(L,cmap = 'gray')
#     plt.title('L'), plt.xticks([]), plt.yticks([])
#     plt.subplot(3,2,3),plt.imshow(H,cmap = 'gray')
#     plt.title('H'), plt.xticks([]), plt.yticks([])
#     plt.subplot(3,2,4),plt.imshow(Ha,cmap = 'gray')
#     plt.title('Ha'), plt.xticks([]), plt.yticks([])
#     plt.subplot(3,2,5),plt.imshow(S,cmap = 'gray')
#     plt.title('Sharp'), plt.xticks([]), plt.yticks([])
#     plt.subplot(1,2,2),plt.imshow(O,cmap = 'gray')
#     plt.title('Output'), plt.xticks([]), plt.yticks([])
#     plt.savefig('cat_Q4_Plot.png') 
#     plt.show()
#----------------------------------------------------------------------------#
def lab1(I):
#     q1(I)
#     q2(I)
#     q3(I)
    q4(I)
#----------------------------------------------------------------------------#    
I = cv2.imread('cat.png', cv2.IMREAD_GRAYSCALE)
start_time = time.time()
lab1(I)
print('Pixels:', I.shape,'Time:', GetTime(time.time() - start_time))
#note - to run all 4 questions, uncomment q1,2,3 in function lab1

S Min: 13
S Max: 171
O Min: 0
O Max: 255

S Min: 1
S Max: 193
O Min: 0
O Max: 255

S Min: -1
S Max: 194
O Min: 0
O Max: 255

Pixels: (460, 600) Time: 0.2 secs
